# Using Python to analyze the Covid19 data
Hong Qin

Code at https://github.com/hongqin/python-covid19-analysis-sandbox/blob/master/PD_demo_jhu_covid19.ipynb

Video at https://youtu.be/1JDP4o92tss 

These materials are for education only.


In [ ]:
### Import libraries
import pandas as pd # primary data structure library

# Part1. Downloading GitHub Data set 

In [ ]:
! git clone https://github.com/CSSEGISandData/COVID-19.git

This data is collected from repository for the 2019 Novel Coronavirus Visual Dashboard operated by the Johns Hopkins University Center for Systems Science and Engineering (JHU CSSE).https://github.com/CSSEGISandData/COVID-19

In [ ]:
### Select data path
fullpath1= '/content/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'

df1 = pd.read_csv(fullpath1, error_bad_lines=False)

fullpath2 = '/content/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'

df2 = pd.read_csv(fullpath2, error_bad_lines=False)

# Part 2. Explore USA time series by counties

In [ ]:
df1.shape

view the dataframe

In [ ]:
df1.head()

How to find out the columns?

In [ ]:
df1.columns

In [ ]:
df1.tail()

In [ ]:
df1['Admin2'][3335:3340]

In [ ]:
df1.iloc[3335:3340, 4:7]

How to find out dataframe dimensions

In [ ]:
print ('dataframe dimensions:', df1.shape)

How many "province states" are there in this data set?

In [ ]:
df1.Province_State.unique()

In [ ]:
len(df1.Province_State.unique())

### Q, what is the larged case value in this data set? 

In [ ]:
df1.iloc[:, 11:].max().max()

Find out the largest value for column '1/14/21'

In [ ]:
df1["1/14/21"].max()

## Find out which row has the largest value. 

In [ ]:
df1.loc[df1['1/14/21']==976075]

In [ ]:
LA = df1.iloc[213, 11:]
type(LA)
#print(LA)
LA.plot()

## plot
Because pandas do plot by columns, JHU data set put time series by row. So, we have to transpose the JHU dataframe. 

In [ ]:
UScases = df1.iloc[0:3, 11:].transpose() 
UScases.shape
UScases

In [ ]:
UScases.plot()
UScases.plot(subplots=True)

In [ ]:
df1.iloc[1:20, 11:].transpose().plot()


## Add meaningful index to the dataframe

In [ ]:
# one way is to load csv with index 
df1 = pd.read_csv(fullpath1, error_bad_lines=False, index_col=10) #10th index picks 11th column of "Combined_Key"
df1.reset_index(drop=True)
df1.index

In [ ]:
df1.shape

In [ ]:
df1.head()

In [ ]:
# another way is to re-assign index
df1 = pd.read_csv(fullpath1, error_bad_lines=False)
newrows = list( df1['Combined_Key'])
df1.index =  newrows
df1.shape

In [ ]:
df1.head()

In [ ]:
list(df1['Combined_Key'] )[1:10]

In [ ]:
step = 10
for i in range(0, 100, step):
#for i in range(0, 3340, step):
  UScases = df1.iloc[ i : (i+step), 11:].transpose()
  UScases.plot()

In [ ]:
df1.tail()

## Note. It turns out that there are unassigned cases in some states, such as Wyoming. Thse unassigned cases become zero when they are settled. 

In [ ]:
df1.loc['Unassigned, Wyoming, US', ][11:].plot()

In [ ]:
df1.loc[ df1.index.str.contains('Unassigned'), ].shape
df1.loc[ df1.index.str.contains('Unassigned'), ].head()

In [ ]:
# df1.loc[df1.index.str.contains('Unassigned'), ].plot() #No, this is not working properly

In [ ]:
# df1.loc[df1.index.str.contains("Unsassigned"),11:].plot() #error 

In [ ]:
# df1.loc[df1.index.str.contains("Unassigned")][11:].plot() #wrong plot

In [ ]:
#df1.loc[df1.index.str.contains('Unassigned'), :][:,11:]

In [ ]:
unassigned_df = df1.loc[df1.index.str.contains('Unassigned')]
unassigned_df.iloc[ :, 11:].shape
unassigned_df.iloc[ :, 11:].transpose().plot()

In [ ]:
df1.loc[df1.index.str.contains('Unassigned')].iloc[:, 11:].transpose().plot() # Well done, a single line solution! 

## How about the cases at the Hamilton county, TN? 

In [ ]:
# Can we examine cumualted cases at the Hamilton County, TN? 


df1.loc[df1['Combined_Key'] == 'Hamilton, Tennessee, US']

In [ ]:
dfHamilton = df1[df1.Combined_Key == "Hamilton, Tennessee, US"]
dfHamilton

In [ ]:
df1.loc[df1['Combined_Key'] == 'Hamilton, Tennessee, US'].iloc[:, 11:].transpose().plot()

#Part 3. Examine the cases in each state.

In [ ]:

df1.loc[ :, 'Province_State']

In [ ]:
groupby_state = df1.iloc[:,11:].groupby(df1['Province_State'])
type( groupby_state )

In [ ]:
df_by_state = groupby_state.sum()
type( df_by_state )

In [ ]:
df_by_state.head()

In [ ]:
df_by_state.transpose().plot()

In [ ]:
df_by_state.transpose().iloc[:,0:5].plot(subplots=True)

In [ ]:
df_by_state.transpose().loc[:,"Tennessee"].plot(subplots=True)

In [ ]:
df_by_state.head()

In [ ]:
# df_by_state.loc['Tennessee'][11:].transpose().plot() #This is bit problematic on the columns

In [ ]:
df_by_state.loc[['Georgia', 'Texas', 'Alabama', 'Florida', 'Tennessee']].transpose().plot() #Here, Texas and Tennessee are row-index

In [ ]:
df_by_state.transpose()[['Texas', 'Tennessee']].plot() #Here, Texas ans Tennessee are columns

# How to examine the new cases daily? 

## diff() on a test dataframe

In [ ]:
dataset = [(2, 4, 6, 8),
           (10, 12, 14, 18),
           (20, 22, 24, 26),
           (28, 30, 32, 34)];

df_test = pd.DataFrame(dataset, columns = ("A", "B", "C", "D" ));


In [ ]:
df_test.diff(axis=0) #by row

In [ ]:
df_test.diff(axis=1) #by column

## diff() on US covid19 data set
 JHU cases are by row, and we need to calculate difference between columns

In [ ]:
df_by_state.head()

In [ ]:
df_dailycases_by_state =  df_by_state.diff(axis=1) #1 by columns
df_dailycases_by_state.head()

In [ ]:
df_dailycases_by_state.transpose()[['Tennessee', 'Wisconsin']].plot()

In [ ]:
df_dailycases_by_state.loc[['Tennessee', 'Texas']].iloc[:, 2:].rolling(window=3).mean()

#.rolling(window=3).mean()

In [ ]:
df_dailycases_by_state.transpose()[['Tennessee', 'Alaska']].rolling(window=3, center=False).mean()

In [ ]:
df_dailycases_by_state.transpose()[['Tennessee', 'Alaska']].head()

In [ ]:
df_dailycases_by_state.transpose()[['Tennessee', 'Alaska', 'Florida']].rolling(window=14).mean().plot(logy=True)

In [ ]:
df_dailycases_by_state.transpose()[['Tennessee', 'Alaska']].rolling(window=3).mean().plot(subplots=True)

In [ ]:
df_dailycases_by_state.transpose()[['Tennessee', 'Texas']].rolling(window=7).mean().plot()

In [ ]:
df_dailycases_by_state.transpose()[['Tennessee', 'Texas']].head()

## Examine the daily confirmed cases in Hamilton County, TN

In [ ]:
df1.loc[df1['Combined_Key'] == 'Hamilton, Tennessee, US'].iloc[:, 11:].transpose().plot()

In [ ]:
df_Hamilton = df1.loc[df1['Combined_Key'] == 'Hamilton, Tennessee, US'].iloc[:, 11:]

type(df_Hamilton)
df_Hamilton.head()

In [ ]:
df_Hamilton.transpose().diff(axis=0).plot()

In [ ]:
df_Hamilton.transpose().diff(axis=0).rolling(window=3).median().plot()

In [ ]:
df_Hamilton.transpose().diff(axis=0).rolling(window=7).mean().plot()

# Pick a time window using datetime

In [ ]:
#from datetime import datetime 

df_dailycases_by_state2 = df_dailycases_by_state.transpose()

df_dailycases_by_state2.head()

In [ ]:
df_dailycases_by_state2.index = pd.to_datetime(df_dailycases_by_state2.index)
df_dailycases_by_state2.index

In [ ]:
df_dailycases_by_state2.loc['2020-08-01':, ['Alabama', 'Tennessee', 'Georgia', 'Virginia', 'Florida']].plot()

## Examine the row with the largest value on 9/8/2020

In [ ]:
subset = df1.loc[df1['9/8/20']==249241]

In [ ]:
subset = subset.transpose()
subset = subset.iloc[:][11:]
type(subset)

In [ ]:
type(subset)

In [ ]:
from datetime import datetime

#datetime.strptime( '9/8/20', '%m/%d/%y') 

dates = [datetime.strptime( x, '%m/%d/%y') for x in subset.columns[11:]]
print(dates)

In [ ]:
df1.iloc[5:10, -20:-1 ]

#Part 4 Explore Global Data set

In [ ]:

df2 = pd.read_csv(fullpath2, error_bad_lines=False)
df2.columns

In [ ]:
df2['Country/Region']
list(df2['Country/Region'].unique())

In [ ]:
# Countries affected
countries = df2['Country/Region'].unique().tolist()
print("\nTotal countries affected by virus: ",len(countries))
print(countries)

In [ ]:
df2by_country = df2.iloc[:, 5:].groupby(df2['Country/Region']).sum().transpose()

In [ ]:
df2by_country.head()

In [ ]:
df2by_country.index = pd.to_datetime(df2by_country.index)

In [ ]:
### Check if the dataframe contains NaN values
df2by_country.head()

In [ ]:
  df2by_country.columns[ df2by_country.columns.str.contains('US') ]

In [ ]:
  df2by_country[['US', 'France', 'India', 'Sweden']].plot(logy=True)


In [ ]:
  df2by_country[['US', 'France', 'India', 'Sweden']]['2020-05-01':].plot(logy=False)

# Alternative approach to load data from GitHub

Go to GitHub, select the csv file,  select "view as raw",  and copy the URL. 

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"

df = pd.read_csv(url)

In [ ]:
df.head

In [ ]:
? pd.read_csv()

In [ ]:
df2 = pd.read_csv(url, skiprows=5)
df2.head

#Part 5. In progress Load Google Mobility data set

In [ ]:
#? not working
urlGoogleMobility = "https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv"
df.GM = pd.read_csv(urlGoogleMobility )

In [ ]:
df.GM.head